<a href="https://colab.research.google.com/github/DevarshDamoDaran/Black_pepper_plant_disease_detection/blob/main/Plant_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python packages and Image Dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8

In [ ]:
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten 
from tensorflow.keras.models import Model 
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.applications.resnet50 import preprocess_input 
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img , img_to_array
from tensorflow.keras.models import Sequential
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import glob as glob 
import os
import cv2
import io
from PIL import Image
import gradio as gr

In [ ]:
train_datagen = ImageDataGenerator(shear_range = 0.3,
                                   zoom_range = 0.5,
                                   horizontal_flip=True,
                                   preprocessing_function=preprocess_input )
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input  )
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/pepperleaf_data/train',
                                                 target_size = (256, 256),
                                                 batch_size = 32)

val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/pepperleaf_data/val',
                                            target_size = (256, 256),
                                            batch_size = 32)

Found 654 images belonging to 3 classes.
Found 120 images belonging to 3 classes.


# Data visualization

In [ ]:
#@title
len(os.listdir("/content/drive/MyDrive/pepperleaf_data/val"))

In [ ]:
#@title
t_img , label = training_set.next()

In [ ]:
#@title
t_img.shape

In [ ]:
#@title
def plotImage(img_arr, label):
  for im, l in zip(img_arr , label):
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.show()

In [ ]:
#@title
plotImage(t_img[:3],label[:3])

# Model Building

In [ ]:
resnet = ResNet50(input_shape=(256,256,3), weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
for layer in resnet.layers:
  layer.trainble = False

In [ ]:
resnet.summary()

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(units= 3, activation='softmax')(x)

# Create a model object 
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam', 
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stoppinh
es = EarlyStopping(monitor= 'val_accuracy', min_delta= 0.01, patience= 3, verbose=1)

# model check point
mc = ModelCheckpoint(filepath="best_model.h5",
                     monitor= 'val_accuracy', 
                     min_delta= 0.01, 
                     patience= 3, 
                     verbose=1, 
                     save_best_only= True)

cb = [es , mc]

In [ ]:
hi = model.fit_generator(
    training_set,
    validation_data = val_set,
    epochs = 20,
    steps_per_epoch = len(training_set),
    validation_steps = len(val_set),
    verbose = 1
)

<ipython-input-11-0f4deab54562>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hi = model.fit_generator(


Epoch 1/20
21/21 [==============================] - 287s 12s/step - loss: 5.7629 - accuracy: 0.7034 - val_loss: 1556311.2500 - val_accuracy: 0.3333
Epoch 2/20
21/21 [==============================] - 17s 822ms/step - loss: 0.9033 - accuracy: 0.8670 - val_loss: 1233811.5000 - val_accuracy: 0.3333
Epoch 3/20
21/21 [==============================] - 17s 825ms/step - loss: 1.1131 - accuracy: 0.8869 - val_loss: 17240006.0000 - val_accuracy: 0.3333
Epoch 4/20
21/21 [==============================] - 18s 845ms/step - loss: 1.6412 - accuracy: 0.8563 - val_loss: 605969.6250 - val_accuracy: 0.3333
Epoch 5/20
21/21 [==============================] - 18s 846ms/step - loss: 0.4331 - accuracy: 0.8853 - val_loss: 23678.7891 - val_accuracy: 0.3417
Epoch 6/20
21/21 [==============================] - 18s 829ms/step - loss: 0.6257 - accuracy: 0.8440 - val_loss: 1448.3153 - val_accuracy: 0.3333
Epoch 7/20
21/21 [==============================] - 19s 896ms/step - loss: 0.2576 - accuracy: 0.9159 - val_loss:

# Prediction and Output

In [ ]:
from keras.models import load_model

model = load_model("/content/drive/MyDrive/model_resnet50.h5")

In [ ]:
acc = model.evaluate_generator(val_set)[1]

print(f"The accuracy of your model is {acc*100} %")

<ipython-input-8-0b0833182562>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc = model.evaluate_generator(val_set)[1]


The accuracy of your model is 99.16666746139526 %


In [ ]:
def prediction(path):

  ref = dict(zip(list(training_set.class_indices.values()), list(training_set.class_indices.keys())))

  img = load_img(path,target_size=(256,256))

  i = img_to_array(img)

  im = preprocess_input(i)

  img = np.expand_dims(im, axis=0)
  
  pred = np.argmax(model.predict(img))
  
  pr = ref[pred]
  #print(f"the image belongs to { ref[pred]}")

  return {pr}

In [ ]:
def predictions(img):
  ref = dict(zip(list(training_set.class_indices.values()), list(training_set.class_indices.keys())))
  img_4d=img.reshape(-1,256,256,3)
  pred = model.predict(img_4d)
  predict = np.argmax(pred)
  pr = ref[predict]
  return pr

In [ ]:
img_path = "/content/drive/MyDrive/pepperleaf_data/val/black_pepper_yellow_mottle_virus/1455-10091-10258.jpg"

prediction(img_path)

1/1 [==============================] - 0s 243ms/step


{'black_pepper_yellow_mottle_virus'}

In [ ]:
image = gr.Image(source="webcam", tool=None, shape=(256,256))
label = gr.Textbox()

gr.Interface(fn=predictions, inputs=image, outputs=label).launch(debug='True')